# オンライン分析用  

受講生のノートブックの実行状況を可視化します。  
各受講生の実行履歴は各ノートブックと同じディレクトリ内の`.log`ディレクトリ以下に出力されており、これを教師が閲覧可能なディレクトリ以下のDB（`/home/{userid}/nbgrader/{course}/exec_history.db`）に収集しています。  
本ノートブックは、そのDBの情報をグラフで表示するためのノートブックです。

## 前提条件

### ノートブックの実行時、カーネルに[LC_wrapper](https://github.com/NII-cloud-operation/Jupyter-LC_wrapper)を利用していること

本ノートブックで利用する実行履歴は、[LC_wrapper](https://github.com/NII-cloud-operation/Jupyter-LC_wrapper)によって収集されたログを利用します。したがって、受講生がノートブックを実行する際に利用するカーネルは[LC_wrapper](https://github.com/NII-cloud-operation/Jupyter-LC_wrapper)である必要があります。

### 課題配布時に、各セルのIDが一意になっていること

ノートブックの各セルを一意に識別するために、[LC_wrapper](https://github.com/NII-cloud-operation/Jupyter-LC_wrapper)によって付与される`meme_id`を利用します。
同じノートブックをコピーして利用した場合など、`meme_id`に枝番が付与されていると、各ユーザの実行情報が正常に取得できません。  
そのような場合、`meme_id`を振り直す必要があります。  
新規に作成したノートブックであっても、一度配布直前に`meme_id`を振り直すことを推奨します。  

- meme_idを振り直すコード例  
ターミナルを起動し、以下のコードを課題ファイル（`.ipynb`）があるディレクトリ（`/home/ユーザID/nbgrader/コース名/source/課題名`）で実行してください。  

```bash
ls *.ipynb | xargs -I {} sh -c 'jupyter nblineage new-root-meme --log-level=DEBUG "{}" "{}_"; mv "{}_" "{}"'
```

### 収集対象  

nbgrader上の受講生となっているユーザが収集対象となります。  
教師ユーザでログイン後、Moodle側で新たに追加されたユーザは、教師ユーザが再度ログインする時まで受講生としてnbgraderに登録されないため、それまでは実行ログが収集されません。

## 表示

出力例を掲載しています。

- ノートブックごとに、表形式で表示します
- 各ユーザごとに、各セルの最新の実行結果が表示されます
  - セルの実行結果が`ok`（正常終了）の場合は背景色緑、セルの実行結果が`error`（異常終了）の場合は背景色赤で表示しています
  - 一度も実行していないか、実行状況を把握できないセルには何も表示されません
![](./images/progress_html.png)

## パラメータ指定

対象のコース等を指定します。

In [ ]:
import os

# nbgrader上で設定した、課題名を入力してください
ASSIGNMENT = 'assign01'

# コース名を環境変数から取得します
COURSE = os.environ['MOODLECOURSE']
DB_PATH = os.path.join(os.environ['HOME'], 'nbgrader', COURSE, 'exec_history.db')

## 出力

出力したHTMLファイルをブラウザで閲覧するか、このノートブック上で閲覧する方法があります。

### html更新用の処理  

htmlファイルに最新のDBの内容を取り込む処理を実装しています。  
ここに定義した関数を実行するたびに、DBから最新の情報を取得し、各ユーザの実行状況をHTMLとして出力します。

In [ ]:
import sqlite3
from jinja2 import Environment, FileSystemLoader

def update_progress_html(fname: str, course: str, assignment: str,
                         html_auto_refresh_sec: int = -1):
    """create or update html
    notebooks = {
      nb_1: [{meme_id: 'aaa', section: '1.2.1', student_results: {'student01': exec_info: {'state': 'ok'}}}]
    }
    """
    file_loader = FileSystemLoader('./')  # template.htmlのパスを指定
    env = Environment(loader=file_loader)
    template = env.get_template('progress.html.j2')
    
    # 最新のログのみ
    sql = """
    SELECT
      notebook_name,
      log_id.section as cell_section,
      ifnull(log_executed.student_id, null) as student_id,
      log_id.id as log_id,
      log_id.section as cell_section,
      ifnull(log_executed.log_execute_reply_status, '') as log_execute_reply_status
    FROM
      log_id left outer join (SELECT
          log.assignment as assignment,
          log.student_id as student_id,
          log.log_id as log_id,
          log_execute_reply_status
        FROM log, 
            (SELECT assignment, student_id, log_id, MAX(log_sequence) as sequence
            FROM log
            WHERE assignment = ?
            GROUP BY
              student_id, log_id) latest_log
        where log.assignment = latest_log.assignment
        and log.student_id = latest_log.student_id
        and log.log_id = latest_log.log_id
        and log.log_sequence = latest_log.sequence
        order by
          log.log_id, student_id) log_executed on log_id.id = log_executed.log_id and log_id.assignment = log_executed.assignment
    WHERE
      log_id.assignment = ?
    ORDER BY
      log_id.notebook_name, log_id.id
    """
    student_sql = """
    SELECT id
    FROM student
    """

    with sqlite3.connect(f"file:{DB_PATH}?mode=ro", uri=True) as con:
        # 列名で結果にアクセスできるようにする
        con.row_factory = sqlite3.Row
        cur = con.cursor()
        cur.execute(sql, [assignment, assignment])
        result = cur.fetchall()
        cur.execute(student_sql)
        students = [d['id'] for d in cur.fetchall()]

    notebooks = dict()
    c_notebook_name = None
    c_log_id = None
    c_idx = 0
    nb_changed = False
    for r in result:
        notebook_name = r['notebook_name']
        log_id = r['log_id']
        cell_section = r['cell_section']
        student_id = r['student_id']
        log_execute_reply_status = r['log_execute_reply_status']

        if c_notebook_name != notebook_name:
            notebooks[notebook_name] = []
            c_notebook_name = notebook_name
            c_idx = 0
            nb_changed = True

        if c_log_id != log_id:
            notebooks[notebook_name].append(dict(meme_id=log_id, section=cell_section))
            c_log_id = log_id
            if nb_changed is True:
                nb_changed = False
                # ノートブックが変わった時はインクリメントしない
            else:
                c_idx += 1
        if 'student_results' not in notebooks[notebook_name][c_idx]:
            notebooks[notebook_name][c_idx]['student_results'] = {}

        if student_id is not None:
            notebooks[notebook_name][c_idx]['student_results'][student_id] = {'exec_info': {'state': log_execute_reply_status}}

    data = {
        'title': '進捗可視化',
        'heading': '進捗可視化',
        'course_name': course,
        'assignment_name': assignment,
        'notebooks': notebooks,
        'users': students,
        'html_auto_refresh_sec': html_auto_refresh_sec,
    }
    output = template.render(data)

    # 結果を出力 (ファイルや標準出力に書き込む)
    with open(fname, 'w') as f:
        f.write(output)

    return fname


## 閲覧  

ブラウザで新たにタブを開いて出力したHTMLを閲覧する方法と、本ノートブックのセル出力でHTMLを閲覧する方法があります。  
自動更新するための手段が異なるため、セルを分けています。

### 別タブで開く場合（自動更新）

HTML自体は自動更新する設定になっていないため、`<meta http-equiv="refresh" content="10" />` を追加して、自動更新するように設定したHTMLを出力します。  
このセルの実行結果として、出力したHTMLファイルへのリンクが表示されます。  
以下のセルでは、10秒ごとに自動でリロードを行う設定になっています。
自動更新する秒数の変更や、自動更新自体行わない場合は、以下のセルを編集してください。  

【補足】  
「1.4.2 セルの出力で開く場合」で表示する場合は、上記自動リロードを設定していると、このノートブックを開いているタブ自体がリロードされてしまいます。これを避けるために、別タブで開く場合と、セルの実行結果で開く場合でhtmlファイルを分けています。  
そのため、htmlファイルの名前を変更する場合は、同じ名前にならないように設定することを推奨します。

In [ ]:
import time
import os
from pathlib import Path

# 自動更新する間隔（秒）
reload_span = 10
html_name = f'{COURSE}_{ASSIGNMENT}_autoreload.html'

f = update_progress_html(html_name, COURSE, ASSIGNMENT, html_auto_refresh_sec=reload_span)
f_path = os.path.join(str(Path().resolve()).replace(os.environ["HOME"] + "/", ""), f)
link = f'https://{os.environ["JUPYTERHUB_FQDN"]}/user/2/files/{f_path}'
print(f"Link: {link}")

while True:
    update_progress_html(html_name, COURSE, ASSIGNMENT, html_auto_refresh_sec=reload_span)
    time.sleep(10)

### セルの出力で開く場合

セルの出力を自動更新するようになっています。

In [ ]:
import time

from IPython.display import HTML

f = update_progress_html(f'{COURSE}_{ASSIGNMENT}.html', COURSE, ASSIGNMENT)
display_handle = display(HTML(f), display_id=True, clear=True)
while True:
    f = update_progress_html(f'{COURSE}_{ASSIGNMENT}.html', COURSE, ASSIGNMENT)
    display_handle.update(HTML(f))
    time.sleep(5)